In [2]:
import ifcopenshell
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary

# Beispielhafte Nutzung
ifc_file_path = '../Hus28_test.ifc'
storey_name = 'Plan 10'

# IFC-Datei öffnen
ifc_file = ifcopenshell.open(ifc_file_path)
print("IFC file opened successfully.")

# Liste, um die GUIDs der Wände im angegebenen Geschoss zu speichern
guids_in_storey = []

# Durchlaufen aller IfcWall-Objekte und deren Beziehungen
for wall in ifc_file.by_type('IfcWall'):
    for rel in ifc_file.by_type("IfcRelContainedInSpatialStructure"):
        if wall in rel.RelatedElements:
            storey = rel.RelatingStructure
            if storey and storey.Name == storey_name:
                guids_in_storey.append(wall.GlobalId)
                break

print(f"Found {len(guids_in_storey)} GUIDs in storey '{storey_name}'.")

# Laden der IFC-Datei
ifc_file = ifcopenshell.open(ifc_file_path)
print("IFC file opened successfully.")

# Erstellen der Topologien aller Wände im IFC-Modell
wall_topologies = Topology.ByIFCFile(file=ifc_file, transferDictionaries=True, includeTypes=['IfcWall'])
print("Initial wall topologies created successfully.")

# Filtern der Topologien nach GUIDs der Wände im Geschoss
walls_in_storey = []

for topo in wall_topologies:
    topo_dict = Topology.Dictionary(topo)
    topo_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
    if topo_guid in guids_in_storey:
        walls_in_storey.append(topo)

print(f"Found {len(walls_in_storey)} wall topologies in storey '{storey_name}'.")



IFC file opened successfully.
Found 138 GUIDs in storey 'Plan 10'.
IFC file opened successfully.
Initial wall topologies created successfully.
Found 138 wall topologies in storey 'Plan 10'.


In [10]:
import re

def filter_ifcspaces_by_storey(spaces, storey_name):
    filtered_spaces = []
    for space in spaces:
        for rel in space.Decomposes:
            if rel.is_a("IfcRelAggregates") and rel.RelatingObject.is_a("IfcBuildingStorey"):
                if rel.RelatingObject.Name == storey_name:
                    filtered_spaces.append(space)
                    break
    return filtered_spaces

def filter_spaces_by_name(spaces):
    digit_pattern = re.compile(r'^\d{1,5}$')
    filtered_spaces = []
    for space in spaces:
        if digit_pattern.match(space.Name):
            filtered_spaces.append(space)
    return filtered_spaces

# Alle IfcSpaces einlesen
spaces = ifc_file.by_type('IfcSpace')

# Nach Stockwerk filtern
spaces_in_storey = filter_ifcspaces_by_storey(spaces, storey_name)
print(f"Filtered {len(spaces_in_storey)} IfcSpaces in storey '{storey_name}'.")

# Nach Namen filtern
filtered_spaces = filter_spaces_by_name(spaces_in_storey)
print(f"Filtered {len(filtered_spaces)} IfcSpaces with valid names.")

Filtered 37 IfcSpaces in storey 'Plan 10'.
Filtered 30 IfcSpaces with valid names.


In [11]:
# Erstellen der Topologien aller gefilterten Räume im IFC-Modell
space_topologies = Topology.ByIFCFile(file=ifc_file, transferDictionaries=True, includeTypes=['IfcSpace'])
print("Initial space topologies created successfully.")

# Filtern der Topologien nach den gefilterten Räumen
spaces_in_storey = []

for topo in space_topologies:
    topo_dict = Topology.Dictionary(topo)
    topo_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
    if any(space.GlobalId == topo_guid for space in filtered_spaces):
        spaces_in_storey.append(topo)

print(f"Found {len(spaces_in_storey)} space topologies in storey '{storey_name}' with valid names.")

Initial space topologies created successfully.
Found 30 space topologies in storey 'Plan 10' with valid names.


In [12]:
# Wall Topologies
print(len(walls_in_storey))

# Wall Topologies
print(len(spaces_in_storey))

# Wall Guids
print(len(guids_in_storey))



138
30
138


In [13]:
from neo4j import GraphDatabase

# Verbindung zur Neo4J-Datenbank herstellen
uri = "bolt://localhost:7687"  # Standardadresse für Neo4J
username = "neo4j"
password = "testpass"
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_wall_nodes(driver, topologies):
    with driver.session() as session:
        for topo in topologies:
            topo_dict = Topology.Dictionary(topo)
            topo_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
            wall_name = Dictionary.ValueAtKey(topo_dict, "IFC_Name")
            wall_type = Dictionary.ValueAtKey(topo_dict, "IFC_Type")
            
            # Erstellen des Cypher-Statements
            cypher_query = """
            MERGE (w:Wall {guid: $guid})
            SET w.name = $name, w.type = $type
            """
            session.run(cypher_query, guid=topo_guid, name=wall_name, type=wall_type)
            print(f"Wall node with GUID {topo_guid} created or updated.")

# Wall Nodes in der Datenbank anlegen
create_wall_nodes(driver, walls_in_storey)


Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdZ created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdx created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdk created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdq created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSda created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nScE created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdY created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nScD created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSc6 created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdj created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nScR created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdW created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSc7 created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSdt created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nSd$ created or updated.
Wall node with GUID 0aNC3YsFzEZf8uB9g1nScJ created or u

In [14]:
# Funktion zum Erstellen von Raum-Nodes in Neo4J
def create_room_nodes(driver, space_topologies):
    with driver.session() as session:
        for topo in space_topologies:
            topo_dict = Topology.Dictionary(topo)
            room_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
            room_name = Dictionary.ValueAtKey(topo_dict, "IFC_Name")
            
            # Erstellen des Raum-Nodes
            cypher_query = """
            MERGE (r:Room {guid: $room_guid})
            SET r.name = $room_name
            """
            session.run(cypher_query, room_guid=room_guid, room_name=room_name)
            print(f"Room node with GUID {room_guid} created or updated.")

# Räume in der Neo4J-Datenbank anlegen
create_room_nodes(driver, spaces_in_storey)

Room node with GUID 0uSXiavKjD892bcKOjMA$f created or updated.
Room node with GUID 0uSXiavKjD892bcKOjMA$x created or updated.
Room node with GUID 2Im_2tj0r41BcHdJK8Kcz3 created or updated.
Room node with GUID 0uSXiavKjD892bcKOjMA$i created or updated.
Room node with GUID 0uSXiavKjD892bcKOjMA$4 created or updated.
Room node with GUID 0uSXiavKjD892bcKOjMA$1 created or updated.
Room node with GUID 0aNC3YsFzEZf8uB9g1nSca created or updated.
Room node with GUID 0aNC3YsFzEZf8uB9g1nSfQ created or updated.
Room node with GUID 0aNC3YsFzEZf8uB9g1nScw created or updated.
Room node with GUID 0aNC3YsFzEZf8uB9g1nScx created or updated.
Room node with GUID 0aNC3YsFzEZf8uB9g1nScv created or updated.
Room node with GUID 3V7W2RscLEExyQLuCnt1p6 created or updated.
Room node with GUID 3V7W2RscLEExyQLuCnt1ow created or updated.
Room node with GUID 3V7W2RscLEExyQLuCnt1ox created or updated.
Room node with GUID 3V7W2RscLEExyQLuCnt1oy created or updated.
Room node with GUID 3V7W2RscLEExyQLuCnt1o$ created or u

In [15]:
# Funktion zum Erstellen von Material Nodes und Beziehungen in Neo4J
def create_material_nodes_and_relationships(driver, wall_topologies):
    wall_materials_dict = {}  # Dictionary zum Speichern der Materialien pro Wand

    with driver.session() as session:
        for topo in wall_topologies:
            topo_dict = Topology.Dictionary(topo)
            wall_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")

            # Open IFC file again to extract material layers
            wall = ifc_file.by_guid(wall_guid)
            if not wall:
                print(f"Wall with GUID {wall_guid} not found.")
                continue

            material_names = []

            # Extract materials associated with the wall
            for rel in wall.HasAssociations:
                if rel.is_a('IfcRelAssociatesMaterial'):
                    material_constituent_set = rel.RelatingMaterial
                    if material_constituent_set.is_a('IfcMaterialConstituentSet'):
                        for constituent in material_constituent_set.MaterialConstituents:
                            material_name = constituent.Material.Name if constituent.Material else "No material name"
                            material_names.append(material_name)

            # Füge die Materialien dem Dictionary hinzu
            wall_materials_dict[wall_guid] = material_names

            if material_names:
                print(f"Materials for wall {wall_guid}: {material_names}")

                # Create Material nodes and relationships for each material layer
                for i, material_name in enumerate(material_names):
                    # Erstelle einen eindeutigen Materialnamen pro Wand
                    unique_material_name = f"{wall_guid}_{material_name}_{i}"

                    # Create unique Material node
                    cypher_query = """
                    MERGE (m:Material {unique_name: $unique_material_name})
                    SET m.Material = $material_name
                    """
                    session.run(cypher_query, unique_material_name=unique_material_name, material_name=material_name)

                    # Create relationship between Wall and Material
                    cypher_query = """
                    MATCH (w:Wall {guid: $wall_guid}), (m:Material {unique_name: $unique_material_name})
                    MERGE (w)-[:ConsistsOf]->(m)
                    """
                    session.run(cypher_query, wall_guid=wall_guid, unique_material_name=unique_material_name)

                    # Create a one-way InternallyConnected relationship between adjacent materials
                    if i > 0:
                        previous_material_name = f"{wall_guid}_{material_names[i-1]}_{i-1}"
                        cypher_query = """
                        MATCH (m1:Material {unique_name: $previous_material_name}), (m2:Material {unique_name: $current_material_name})
                        MERGE (m1)-[:InternallyConnected]->(m2)
                        """
                        session.run(cypher_query, previous_material_name=previous_material_name, current_material_name=unique_material_name)
            else:
                print(f"No materials found for wall {wall_guid}.")

    return wall_materials_dict  # Rückgabe des Dictionaries

# Materialien und Beziehungen zur Neo4J-Datenbank hinzufügen
wall_materials_dict = create_material_nodes_and_relationships(driver, walls_in_storey)

# Überprüfung des erstellten Dictionaries
print("Wall materials dictionary:")
for wall_guid, materials in wall_materials_dict.items():
    print(f"Wall {wall_guid}: {materials}")

Materials for wall 0aNC3YsFzEZf8uB9g1nSdZ: ['Hemlingby-BrandGips', 'Hemlingby-KL-Trä', 'Air Gap/Luftspalt']
Materials for wall 0aNC3YsFzEZf8uB9g1nSdx: ['Hemlingby-Gips', 'Hemlingby-KL-Trä', 'Hemlingby-Gips']
Materials for wall 0aNC3YsFzEZf8uB9g1nSdk: ['W_Skiffer', 'Hemlingby-Läkt', 'Hemlingby-Plåtläkt', 'Hemlingby-Rockwool red-air', 'Hemlingby-KL-Trä']
Materials for wall 0aNC3YsFzEZf8uB9g1nSdq: ['Hemlingby-Gips', 'Hemlingby-Gips', 'Hemlingby-95 Regel', 'W_Default Walls', 'Hemlingby-95 Regel', 'Hemlingby-Gips', 'Hemlingby-Gips']
Materials for wall 0aNC3YsFzEZf8uB9g1nSda: ['Hemlingby-Gips', 'Hemlingby-KL-Trä', 'Hemlingby-Gips']
Materials for wall 0aNC3YsFzEZf8uB9g1nScE: ['W_Skiffer', 'Hemlingby-Läkt', 'Hemlingby-Plåtläkt', 'Hemlingby-Rockwool red-air', 'Hemlingby-KL-Trä']
Materials for wall 0aNC3YsFzEZf8uB9g1nSdY: ['Hemlingby-BrandGips', 'Hemlingby-KL-Trä', 'Air Gap/Luftspalt']
Materials for wall 0aNC3YsFzEZf8uB9g1nScD: ['W_Skiffer', 'Hemlingby-Läkt', 'Hemlingby-Plåtläkt', 'Hemlingby-Roc

In [16]:
import ifcopenshell
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from neo4j import GraphDatabase

# Funktion zur Überprüfung von Berührungen zwischen Zelle und Räumen
def check_touching_spaces(cell, space_topologies):
    for space_topo in space_topologies:
        merged_topo = Topology.Merge(cell, space_topo)
        shared_faces = Topology.SharedFaces((Topology.Cells(merged_topo)[0]), (Topology.Cells(merged_topo)[1]))
        if shared_faces:
            return Dictionary.ValueAtKey(Topology.Dictionary(space_topo), "IFC_guid")
    return None

# Funktion zur Erstellung von "IsFacing"-Beziehungen in Neo4J
def add_is_facing_relationships(driver, wall_topologies, space_topologies, wall_materials_dict):
    with driver.session() as session:
        for wall_topo in wall_topologies:
            topo_dict = Topology.Dictionary(wall_topo)
            wall_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")

            # Materialliste für die aktuelle Wand abrufen
            materials = wall_materials_dict.get(wall_guid, [])
            if not materials:
                print(f"No materials found for wall {wall_guid}.")
                continue

            # Zellen der Wandtopologie abrufen und invertieren
            wall_cells = Topology.Cells(wall_topo)[::-1]

            if len(wall_cells) < 2:
                print(f"Wall with GUID {wall_guid} does not have enough cells for processing.")
                continue

            # Erste (äußerste) und letzte (innerste) Zelle in der invertierten Liste
            outer_cell = wall_cells[0]
            inner_cell = wall_cells[-1]

            # Berührungen für äußere und innere Schicht prüfen und Beziehungen erstellen
            def create_is_facing_relationship(cell, material_name, layer_index):
                space_guid = check_touching_spaces(cell, space_topologies)
                if space_guid:
                    unique_material_name = f"{wall_guid}_{material_name}_{layer_index}"
                    cypher_query = """
                    MATCH (m:Material {unique_name: $material_name}), (r:Room {guid: $space_guid})
                    MERGE (m)-[:IsFacing]->(r)
                    """
                    session.run(cypher_query, material_name=unique_material_name, space_guid=space_guid)
                    print(f"Added IsFacing relationship between Material {unique_material_name} and Room {space_guid}")

            # Erstellen der Beziehung für die äußere Schicht (Index 0)
            create_is_facing_relationship(outer_cell, materials[0], 0)

            # Erstellen der Beziehung für die innere Schicht (Index n-1)
            create_is_facing_relationship(inner_cell, materials[-1], len(materials) - 1)

# Nutzung der Funktion zum Hinzufügen von IsFacing Beziehungen
add_is_facing_relationships(driver, walls_in_storey, spaces_in_storey, wall_materials_dict)


Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdZ_Hemlingby-BrandGips_0 and Room 2_G3CYmpjD1R88WI3Xz4q$
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdx_Hemlingby-Gips_0 and Room 0aNC3YsFzEZf8uB9g1nSfO
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdx_Hemlingby-Gips_2 and Room 0aNC3YsFzEZf8uB9g1nSfO
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdk_W_Skiffer_0 and Room 0uSXiavKjD892bcKOjMA$x
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdq_Hemlingby-Gips_0 and Room 0aNC3YsFzEZf8uB9g1nSfO
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdq_Hemlingby-Gips_6 and Room 0aNC3YsFzEZf8uB9g1nSfO
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSda_Hemlingby-Gips_2 and Room 0aNC3YsFzEZf8uB9g1nSfO
Added IsFacing relationship between Material 0aNC3YsFzEZf8uB9g1nSdY_Hemlingby-BrandGips_0 and Room 0aNC3YsFzEZf8uB9g1nSfQ
Added IsFacing relationship between Material 0aNC3YsFzEZf8u

In [17]:
# Treiber schließen
driver.close()